In [1]:
import datetime
import fetcher
import compute
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2020, 1, 1)
endDate = datetime.date(2024, 8, 13)
#endDate = datetime.date.today()
Stocks_list = []#"NVDA", "AAPL", "DKNG", "UBER"

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS\\US_2024-08-13.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]
#stocks = stocks[~stocks["Industry"].isin(["Biotechnology", "Pharmaceuticals: major", "Medical Specialties"])]
for ticker in stocks["Symbol"].tolist()[1:]:
    Stocks_list.append(ticker)

df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

leng = 252

focus = pd.DataFrame(columns=['ticker','ADV'])
for ticker in Stocks_list:
    
    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")

    if len(df) < leng:
        continue
    
        
    else:
        # Avg Volume*Price
        ADV = compute.avg_dollar_vol(df, 30)
        new_row = {"ticker":ticker, "ADV": ADV}
        focus = focus.append(new_row, ignore_index=True)

   



### Filtering

In [2]:
final_focus = focus.loc[ (focus['ADV'] >=10000000) ]
len(final_focus["ticker"].values)

2039

### Sorting by ADV

In [3]:
final_focus = pd.DataFrame(final_focus.sort_values(by=['ADV'], ascending=False).to_numpy(), 
                   index=final_focus.index, columns=final_focus.columns)

### Grouping by Sectors & Industries

In [4]:
sectors = {}
for index, row in final_focus.iterrows():
    exchange = stocks.loc[stocks['Symbol'] == row["ticker"]]["Exchange"].values[0]
    sector = stocks.loc[stocks['Symbol'] == row["ticker"]]["Sector"].values[0]
    #print(sector)
    industry = stocks.loc[stocks['Symbol'] == row["ticker"]]["Industry"].values[0]
    #print(industry)
    if sector not in sectors.keys():
        sectors[sector] = {}
        
    if industry not in sectors[sector].keys():
        #print(industry)
        sectors[sector][industry] = []
    
    #print(sectors)
            #print(sectors[sector])
    sectors[sector][industry].append(row["ticker"])

### Plotting RS line of Industries w.r.t realtive stock and comparing with the relative stock

In [5]:
import plotly.graph_objects as go
import numpy as np

industry_index = {}
for i in range(len(sectors.values())):
    for key, values in list(sectors.values())[i].items():
        if len(values)>=20:
            stocks_list = values[0:20]
        else:
            stocks_list = values
        df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")
        df_index = df_index[-1*leng:]

        total_ADV = 0
        total_MCap = 0
        weighted_sum_close_ADV = 0
        weighted_sum_close_MCap = 0
        for ticker in stocks_list:
            #ticker = ticker.split(":")[1]
            #Neglecting tickers with .A or .B or "/" in their names
            if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
                continue
            df = fetcher.__download_data([ticker], startDate, endDate, "US")
            df = df[-1*leng:]
            #MarkerCap to use here is based on float% (Most etfs weight according to this)
            market_cap = stocks.loc[stocks['Symbol'] == ticker]["Market capitalization"].values[0]
            float_shares_perc = stocks.loc[stocks['Symbol'] == ticker]["Float shares %"].values[0]

            market_cap = market_cap*float_shares_perc
            ADV = focus.loc[focus['ticker'] == ticker]["ADV"].values[0]
            total_ADV += ADV 
            total_MCap += market_cap
            weighted_sum_close_ADV += ADV*df["Close"]#[-252:0]
            weighted_sum_close_MCap += market_cap*df["Close"]
        weighted_sum_close_ADV /= total_ADV
        RS_ADV = weighted_sum_close_ADV.values/df_index["Close"]
        weighted_sum_close_MCap /= total_MCap
        RS_MCap = weighted_sum_close_MCap.values/df_index["Close"]

        #Saving the index of each industry
        industry_index[key]=weighted_sum_close_ADV
        #################################################################################
        """#Normalization  ( To directly compare slopes of two lines slope1_normalized and slope2 since both are in same scale)
        lower = min(RS_ADV)
        upper = max(RS_ADV)

        _min = min(df_index["Close"])
        _max = max(df_index["Close"])
        df_index_normalized = lower + ((df_index["Close"] - _min)/(_max - _min))*(upper-lower)
        x = list(range(0, leng)) #x = df_index["Formatted Date"] #list(range(1, leng))
        y1_normalized = df_index_normalized

        # Select the last 20 data points for regression
        x_last_20 = x[-20:]
        y1_normalized_last_20 = y1_normalized[-20:]

        # Perform linear regression for the last 20 points
        slope1_normalized, intercept1_normalized = np.polyfit(x_last_20, y1_normalized_last_20, 1)
        """
        #######################################################################################


        

### Plotting weighted_sum_close of all the industries with SMAs

In [8]:
industries_to_concentrate = []
def SMA(data, window):
    sma = np.convolve(data, np.ones(window), 'valid') / window
    padded_sma = np.concatenate((np.full(window-1, np.nan), sma))
    return padded_sma

for key, index in industry_index.items():
    fig = go.Figure()
    
    x = list(range(0, leng)) #x = df_index["Formatted Date"] #list(range(1, leng))
    y1 = index    # First line (e.g., stock price 1)
    y2 = SMA(index,21) 
    y3 = SMA(index,50)
    y4 = SMA(index,200) 

    x_last_20 = x[-20:]
    y3_last_20 = y3[-20:] 
    y4_last_20 = y4[-20:]

    slopeSMA50, interceptSMA50 = np.polyfit(x_last_20, y3_last_20, 1)
    slopeSMA200, interceptSMA200 = np.polyfit(x_last_20, y4_last_20, 1)
    y5 = df_index["Close"]
    _52weekHigh = max(index)
    if y2[-1] > y3[-1] and y3[-1] > y4[-1] and y1[-1] > y4[-1] and slopeSMA50 >=0 and index[-1] >= .8*_52weekHigh: # and slopeSMA200 >=0 
        industries_to_concentrate.append(key)
    # Add the first line (left y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y1, 
            name="weighted_sum_close",
            yaxis="y1",  # Bind to y-axis 1 (left)
            line=dict(color='blue')
        ))

        # Add the second line (right y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y2, 
            name="21SMA",
            yaxis="y1",  # Bind to y-axis 2 (right)
            line=dict(color='yellow')
        ))

        # Add the linear regression line for y1 (left y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y3, 
            name="50SMA",
            yaxis="y1",  # Bind to y-axis 1 (left)
            line=dict(color='green')
        ))

        # Add the linear regression line for y2 (right y-axis)
        fig.add_trace(go.Scatter(
            x=x, 
            y=y4, 
            name="200SMA",
            yaxis="y1",  # Bind to y-axis 2 (right)
            line=dict(color='red')
        ))

        fig.add_trace(go.Scatter(
            x=x, 
            y=y5, 
            name="SPY",
            yaxis="y2",  # Bind to y-axis 2 (right)
            line=dict(color='gray')
        ))

        # Update layout to include both y-axes
        fig.update_layout(
            title=key,
            xaxis=dict(
                title="X-axis",
                domain=[0.1, 0.9]  # Adjusting the domain to give space for both y-axes
            ),
            yaxis=dict(
                title="Price",
                titlefont=dict(color="blue"),
                tickfont=dict(color="blue")
            ),
            yaxis2=dict(
                    title="Price",
                    titlefont=dict(color="gray"),
                    tickfont=dict(color="gray"),
                    anchor="x",  # Bind this y-axis to the same x-axis
                    overlaying="y",  # Overlay it on the first y-axis
                    side="right"  # Position it on the right
                ),
            
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="center",
                x=0.5
            ),
            hovermode="x unified",
            dragmode="pan",  # Enable panning with left click and drag
        )

        # Enable scroll zoom
        fig.update_layout(
            autosize=True,
        )

        # Activate scroll zooming and disable box zoom
        config = dict(
            scrollZoom=True,  # Enable zooming with mouse scroll
            displayModeBar=True,  # Show the mode bar for interactivity options
            modeBarButtonsToRemove=['zoom2d'],  # Remove the zoom box tool
        )

        # Show the figure
        fig.show(config=config)

In [9]:
industries_to_concentrate

['Telecommunications equipment',
 'Aerospace & defense',
 'Homebuilding',
 'Data processing services',
 'Specialty stores',
 'Home improvement chains',
 'Food retail',
 'Pharmaceuticals: other',
 'Pharmaceuticals: generic',
 'Financial publishing/Services',
 'Managed health care',
 'Hospital/Nursing management',
 'Services to the health industry',
 'Medical/Nursing services',
 'Major banks',
 'Investment banks/Brokers',
 'Investment managers',
 'Finance/Rental/Leasing',
 'Multi-line insurance',
 'Real estate investment trusts',
 'Regional banks',
 'Property/Casualty insurance',
 'Insurance brokers/Services',
 'Life/Health insurance',
 'Real estate development',
 'Specialty insurance',
 'Building products',
 'Metal fabrication',
 'Office equipment/Supplies',
 'Air freight/Couriers',
 'Tobacco',
 'Broadcasting',
 'Publishing: newspapers',
 'Media conglomerates',
 'Publishing: books/magazines',
 'Wireless telecommunications',
 'Specialty telecommunications',
 'Gas distributors',
 'Water u

### Now Calculating all the usual parameters for filtering top stocks

In [25]:
info = {}

for ticker in focus["ticker"]:
    
    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    
   
    # IPO stocks in the last 3 years
    if len(df) >= 252:
        # Avg Volume*Price
        ADV = compute.avg_dollar_vol(df, 30)

        # ADR%
        ADR = compute.ADR(df, 20)
        
        #52 week highs
        _52weekHigh = max(df["Close"][-252:])


        EMA10 = TA.EMA(df, period= 10)
        EMA21 = TA.EMA(df, period= 21)
        SMA50 =  np.convolve(df["Close"], np.ones(50)/50, mode='valid')
        SMA200 = np.convolve(df["Close"], np.ones(200)/200, mode='valid')
        slope50 = compute.linear_regression(SMA50[-5:])
        slope200 = compute.linear_regression(SMA200[-5:])

        info[ticker] = {"Close": df["Close"][-1], "ADR" : ADR, "ADV" : ADV, "52weekHigh":_52weekHigh, "EMA10": EMA10[-1], "EMA21": EMA21[-1], "SMA50": SMA50[-1], "SMA200": SMA200[-1], "Slope50": slope50, "Slope200": slope200}



# Storing all info in a dataframe
new_focus = pd.DataFrame(columns=['ticker',"Close", 'ADR','ADV', '52weekHigh', "EMA10", "EMA21","SMA50", "SMA200", "Slope50", "Slope200"])
for ticker, values in info.items():
    new_row = {"ticker":ticker, "Close" : values["Close"], "ADR": values["ADR"], "ADV": values["ADV"], '52weekHigh' : values["52weekHigh"], "EMA10":values["EMA10"], "EMA21":values["EMA21"], "SMA50": values["SMA50"], "SMA200": values["SMA200"], "Slope50": values["Slope50"], "Slope200": values["Slope200"]}
    new_focus = new_focus.append(new_row, ignore_index=True)

In [26]:
final_final_focus = new_focus.loc[ (new_focus['ADR'] >=2) & (new_focus['ADV'] >=10000000) &(new_focus["Close"] >= new_focus['SMA200'])  & (new_focus['SMA50'] >=new_focus['SMA200']) & (new_focus['Close'] >=.8*new_focus['52weekHigh'])& (new_focus["EMA10"] >= new_focus['SMA50']) & (new_focus["EMA21"] >= new_focus['SMA50'])] #&  & (focus['RS_ratings'] >= 40) & (focus['Slope50'] >=0) & (focus['Slope200'] >=0)


### Top stocks in each industry ( slope of RS w.r.t industry_index >= 0)

In [28]:
new_industries = {}
for i in range(len(sectors.values())):
    for key, values in list(sectors.values())[i].items():
        stocks_list = values

        if key not in industries_to_concentrate:
            continue

        if key not in new_industries.keys():
            new_industries[key] = []

        for ticker in stocks_list:
            if ticker not in final_final_focus["ticker"].values:
                continue
            #ticker = ticker.split(":")[1]
            #Neglecting tickers with .A or .B or "/" in their names
            if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
                continue
            df = fetcher.__download_data([ticker], startDate, endDate, "US")
            df = df[-126:]
            RS = df["Close"]/industry_index[key]

            #################################################################################################################################
            # Example data
            x = list(range(0, 126)) #x = df_index["Formatted Date"] #list(range(1, 126))
            y1 = RS     # First line (e.g., stock price 1)

            # Select the last 20 data points for regression
            x_last_20 = x[-20:]
            y1_last_20 = y1[-20:]

            # Perform linear regression for the last 20 points
            slope1, intercept1 = np.polyfit(x_last_20, y1_last_20, 1)

            

            if slope1 >= 0:
                new_industries[key].append(ticker)

In [29]:
new_industries

{'Aerospace & defense': ['LMT', 'HWM', 'AXON', 'HEI'],
 'Homebuilding': ['DHI', 'LEN', 'FTDR'],
 'Tools & hardware': [],
 'Recreational products': ['HAS', 'VSTO', 'HAYW'],
 'Internet software/Services': ['SPOT', 'IT', 'CHKP', 'MMYT', 'EVER'],
 'Information technology services': ['BR',
  'GWRE',
  'PSN',
  'VRNS',
  'SSNC',
  'CACI',
  'QTWO',
  'BOX'],
 'Data processing services': ['PAYX', 'GDDY', 'TYL', 'EXLS'],
 'Internet retail': ['MELI', 'EBAY', 'RVLV'],
 'Specialty stores': ['CVNA', 'ORLY', 'AZO', 'MUSA', 'GPI', 'RUSHA', 'SAH'],
 'Home improvement chains': ['LOW'],
 'Department stores': [],
 'Drugstore chains': [],
 'Food retail': ['SFM'],
 'Medical specialties': ['TMO',
  'DHR',
  'ISRG',
  'MMM',
  'RMD',
  'HOLX',
  'TMDX',
  'STE',
  'RVTY',
  'QGEN',
  'ICUI',
  'MMSI',
  'CDNA',
  'CSTL'],
 'Pharmaceuticals: other': ['ARGX'],
 'Pharmaceuticals: generic': ['TEVA'],
 'Managed health care': ['UNH'],
 'Hospital/Nursing management': ['UHS'],
 'Services to the health industry': []

In [31]:
dayum = []

for industry in new_industries.values():
    for ticker in industry:
        dayum.append(ticker)

In [32]:
dayum

['LMT',
 'HWM',
 'AXON',
 'HEI',
 'DHI',
 'LEN',
 'FTDR',
 'HAS',
 'VSTO',
 'HAYW',
 'SPOT',
 'IT',
 'CHKP',
 'MMYT',
 'EVER',
 'BR',
 'GWRE',
 'PSN',
 'VRNS',
 'SSNC',
 'CACI',
 'QTWO',
 'BOX',
 'PAYX',
 'GDDY',
 'TYL',
 'EXLS',
 'MELI',
 'EBAY',
 'RVLV',
 'CVNA',
 'ORLY',
 'AZO',
 'MUSA',
 'GPI',
 'RUSHA',
 'SAH',
 'LOW',
 'SFM',
 'TMO',
 'DHR',
 'ISRG',
 'MMM',
 'RMD',
 'HOLX',
 'TMDX',
 'STE',
 'RVTY',
 'QGEN',
 'ICUI',
 'MMSI',
 'CDNA',
 'CSTL',
 'ARGX',
 'TEVA',
 'UNH',
 'UHS',
 'EHC',
 'ENSG',
 'ADUS',
 'PYPL',
 'BFAM',
 'ABM',
 'ICFI',
 'PAYO',
 'MCO',
 'EFX',
 'TRU',
 'CRTO',
 'BLK',
 'MORN',
 'CIGI',
 'CNS',
 'HIG',
 'CINF',
 'AXS',
 'CNO',
 'TRUP',
 'BWIN',
 'CNA',
 'AMT',
 'WELL',
 'PSA',
 'IRM',
 'MAA',
 'CPT',
 'DOC',
 'KIM',
 'AMH',
 'REG',
 'COLD',
 'OHI',
 'FRT',
 'VNO',
 'BRX',
 'FR',
 'STAG',
 'IRT',
 'CTRE',
 'KRG',
 'CUZ',
 'HIW',
 'DEI',
 'ROIC',
 'CDP',
 'AKR',
 'NHI',
 'CB',
 'ALL',
 'WRB',
 'RNR',
 'MTG',
 'THG',
 'PLMR',
 'FCNCA',
 'RYAN',
 'CBRE',
 'FNF',
 'H